In [ ]:
import eddy_footprint as ef
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import xarray as xr

In [ ]:
# Load the data
demo_datapath = "../data/demo.csv"
stable_datapath = "../data/stable_test.csv"
unstable_datapath = "../data/unstable_test.csv"
neutral_datapath = "../data/neutral_test.csv"
df_demo = pd.read_csv(
    demo_datapath, parse_dates=[1], na_values="NA", delimiter=" *, *", index_col=False
)
df_stable = pd.read_csv(
    stable_datapath, parse_dates=[1], na_values="NA", delimiter=" *, *", index_col=False
)
df_unstable = pd.read_csv(
    unstable_datapath,
    parse_dates=[1],
    na_values="NA",
    delimiter=" *, *",
    index_col=False,
)
df_neutral = pd.read_csv(
    neutral_datapath,
    parse_dates=[1],
    na_values="NA",
    delimiter=" *, *",
    index_col=False,
)

# Note:
# eddy-footprint does not QA/QC or u* filter your eddy covariance data.
# bad flux observations should be filtered before calculating footprints
# or, care should be taken to remove or avoid interpretting footprints for bad fluxes

In [ ]:
# create footprints using the Hsieh model for the demo dataset:
demo_foots_Hsieh = ef.calc_footprint(
    air_pressure=df_demo.air_pressure,
    air_temperature=df_demo.air_temperature,
    friction_velocity=df_demo.friction_velocity,
    wind_speed=df_demo.wind_speed,
    cross_wind_variance=df_demo.v_var,
    wind_direction=df_demo.wind_dir,
    monin_obukhov_length=df_demo.L,
    time=df_demo.datetime,
    instrument_height=2.53,
    roughness_length=0.0206,
    workers=-1,
)
# create footprints using the Kormann & Meixner model for the demo dataset:
demo_foots_KM = ef.calc_footprint(
    air_pressure=df_demo.air_pressure,
    air_temperature=df_demo.air_temperature,
    friction_velocity=df_demo.friction_velocity,
    wind_speed=df_demo.wind_speed,
    cross_wind_variance=df_demo.v_var,
    wind_direction=df_demo.wind_dir,
    monin_obukhov_length=df_demo.L,
    time=df_demo.datetime,
    instrument_height=2.53,
    roughness_length=0.0206,
    workers=-1,
    method="Kormann & Meixner",
)
# Note:
# the default for 'workers' is 1, which is no parallelization.
# here we have set 'workers=-1' which implements parallelization
# in the rotations and interpolation steps and runs faster.

In [ ]:
# Stack of demo footprints (one day of half-hourly fluxes from July 12th 2020)
stack_H = demo_foots_Hsieh.sum(dim="time")
stack_KM = demo_foots_KM.sum(dim="time")

In [ ]:
# Plot the demo footprint climatologies.
# Note that xarray objects have full matplotlib functionality.
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(8, 3))

stack_H.plot(ax=axes[0], vmin=0, vmax=0.015)
axes[0].set_xlim([-100, 100])
axes[0].set_ylim([-100, 100])
axes[0].set_title("Hsieh composite 7/12/2020")
axes[0].plot(0, 0, marker=".", color="w")

stack_KM.plot(ax=axes[1], vmin=0, vmax=0.015)
axes[1].set_xlim([-100, 100])
axes[1].set_ylim([-100, 100])
axes[1].set_title("Kormann & Meixner composite 7/12/2020")
axes[1].plot(0, 0, marker=".", color="w")

fig.tight_layout()

In [ ]:
# Plot multiple footprints in the timeseries as a facetgrid

# slice a subset to plot
time_slice = demo_foots_Hsieh.isel(time=slice(0, 6, 1))
time_slice.coords
# make facetgrid
fig_facet = time_slice.plot(x="x", y="y", col="time", col_wrap=3)
# add marker for the tower location at 0,0
fig_facet.map(lambda: plt.plot(0, 0, marker=".", color="w"))
plt.xlim(-250, 250)
plt.ylim(-250, 250)

In [ ]:
# Export xarray dataset as netcdf
netcdf_path = "../data/demo_footprints_Hsieh.nc"
demo_foots_Hsieh.to_netcdf(netcdf_path)

In [ ]:
# create footprints using the Hsieh model for the three test regimes
stable_foots_Hsieh = ef.calc_footprint(
    air_pressure=df_stable.air_pressure,
    air_temperature=df_stable.air_temperature,
    friction_velocity=df_stable.friction_velocity,
    wind_speed=df_stable.wind_speed,
    cross_wind_variance=df_stable.v_var,
    wind_direction=df_stable.wind_dir,
    monin_obukhov_length=df_stable.L,
    time=df_stable.datetime,
    instrument_height=2.53,
    roughness_length=0.0206,
    workers=-1,
)
unstable_foots_Hsieh = ef.calc_footprint(
    air_pressure=df_unstable.air_pressure,
    air_temperature=df_unstable.air_temperature,
    friction_velocity=df_unstable.friction_velocity,
    wind_speed=df_unstable.wind_speed,
    cross_wind_variance=df_unstable.v_var,
    wind_direction=df_unstable.wind_dir,
    monin_obukhov_length=df_unstable.L,
    time=df_unstable.datetime,
    instrument_height=2.53,
    roughness_length=0.0206,
    workers=-1,
)
neutral_foots_Hsieh = ef.calc_footprint(
    air_pressure=df_neutral.air_pressure,
    air_temperature=df_neutral.air_temperature,
    friction_velocity=df_neutral.friction_velocity,
    wind_speed=df_neutral.wind_speed,
    cross_wind_variance=df_neutral.v_var,
    wind_direction=df_neutral.wind_dir,
    monin_obukhov_length=df_neutral.L,
    time=df_neutral.datetime,
    instrument_height=2.53,
    roughness_length=0.0206,
    workers=-1,
)
# create footprints using the Kormann & Meixner model for the two test regimes
stable_foots_KM = ef.calc_footprint(
    air_pressure=df_stable.air_pressure,
    air_temperature=df_stable.air_temperature,
    friction_velocity=df_stable.friction_velocity,
    wind_speed=df_stable.wind_speed,
    cross_wind_variance=df_stable.v_var,
    wind_direction=df_stable.wind_dir,
    monin_obukhov_length=df_stable.L,
    time=df_stable.datetime,
    instrument_height=2.53,
    roughness_length=0.0206,
    workers=-1,
    method="Kormann & Meixner",
)
unstable_foots_KM = ef.calc_footprint(
    air_pressure=df_unstable.air_pressure,
    air_temperature=df_unstable.air_temperature,
    friction_velocity=df_unstable.friction_velocity,
    wind_speed=df_unstable.wind_speed,
    cross_wind_variance=df_unstable.v_var,
    wind_direction=df_unstable.wind_dir,
    monin_obukhov_length=df_unstable.L,
    time=df_unstable.datetime,
    instrument_height=2.53,
    roughness_length=0.0206,
    workers=-1,
    method="Kormann & Meixner",
)
# Note:
# The Kormann & Meixner model should not be used for neutral or near-neutral conditions.

In [ ]:
# Plot the footprints from different atmospheric conditions:
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(12, 3))

stable_foots_Hsieh.isel(time=1).plot(ax=axes[0])
axes[0].set_xlim([-250, 250])
axes[0].set_ylim([-250, 250])
axes[0].plot(0, 0, marker=".", color="w")
axes[0].set_title("Hsieh stable")

unstable_foots_Hsieh.isel(time=1).plot(ax=axes[1])
axes[1].set_xlim([-250, 250])
axes[1].set_ylim([-250, 250])
axes[1].plot(0, 0, marker=".", color="w")
axes[1].set_title("Hsieh unstable")

neutral_foots_Hsieh.isel(time=1).plot(ax=axes[2])
axes[2].set_xlim([-250, 250])
axes[2].set_ylim([-250, 250])
axes[2].plot(0, 0, marker=".", color="w")
axes[2].set_title("Hsieh near neutral")

fig.tight_layout()